# PRÁCTICA 2: Clasificador de noticias

### Nombres:
Introduce en esta celda los nombres de los dos integrantes del grupo:\
*Alumno 1:* DANIEL CARMONA PEDRAJAS
*Alumno 2:* JOEL PARDO FERRERA

Objetivo: Implementar un clasificador usando el conjunto de datos recopilado de varias fuentes de internet como:

- Google News que toma noticias de varios repositorios dedicados a la información
- Periodicos:
    - El pais
    - ABC
    - El confidencial
    - 20minutos
    - El diario

Este repositorio incluye tanto las noticias en formato '.txt' donde se almacenan los cuerpos de noticia y sus correspondientes titulos, como un '.csv' donde se contiene un registro de todas las noticias donde se refleja el número de noticias, la clase a la que pertenece (deportes, salud, ciencia y politica), el número de noticia dentro de la clase correspondiente, el titulo de noticia, la ruta donde esta almacenada esa noticia, y por ultimo la URL correspondiente del repositorio de donde se ha sacado al noticia. 

La fechas fechas tanto de publicacion como de obtenciond e datos se ubican en Noviembre de 2022. 

La clase a predecir es el tipo de noticia (columna 'category' de la base de datos), a partir de los archivos '.txt'. 

## 1. IMPORTACION DE LIBRERIAS

In [187]:
# git config --global user.email "jpardo0824@gmail.com"
# git config --global user.name "JPardo08"

In [188]:
!pip3 install pandas
!pip3 install spacy
!pip3 install scikit-learn

In [189]:
import pandas as pd
import numpy as np
import spacy
import unicodedata
import os
#from spellchecker import SpellChecker 
#from textblob import TextBlob 
#import contractions
import re
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC

## 2. CARGA DE DATOS
Cargamos los datos en dos formatos:
* DataFrame de pandas
* Generador

Para cargar los datos utilizamos la libreria Pandas. En la funcion impleentada, le pasamos unicmanete  la ruta del archivo csv que queramos cargar y se guarda los datos en una variable generador.

In [190]:
ruta = "/Users/joelpardo/Desktop/TextClassification" ## CAMBIAR

In [191]:
def txt(df):
    """ Funcion para coger los documentos '.txt' de las noticias
    
    Path: Path de la carpeta donde se encuentras los documentos
    """


    paths = df["path"].tolist()
    clases = df["category"].unique().tolist()

    documentos = []

    for i in paths:
        
        t = i.replace(".", ruta,1) 
        s = t.replace("/", "//")
        print(s)

        with open(s, "r", encoding="utf-8", errors='ignore') as f:
            
            lineas = f.readlines()
            txt1 = ''.join(lineas)
            documentos.append(txt1)
            



    
    return clases, documentos




In [192]:
p_df = ruta + "/Datos/urls1.csv"
p_df2 = p_df.replace("/", "//")
print(p_df2)

# df_gen = parse(r'/Users/joelpardo/Desktop/TextClassification/Datos/urls1.csv')



df = pd.read_csv(p_df2,',')

#Visualizamos los datos
df.head()

//Users//joelpardo//Desktop//TextClassification//Datos//urls1.csv


/var/folders/yq/prfd22yn0k3gnlt89mfz3zb80000gn/T/ipykernel_2913/261014751.py:9: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(p_df2,',')


,index,category,n_doc,title,path,link
0,0,sports,1,Memphis y Dumfries certifican que el 'soccer' ...,./Datos/Raw_data/sports/1.txt,https://news.google.com/__i/rss/rd/articles/CB...
1,1,sports,2,Con el dinero no basta en el fútbol - AS,./Datos/Raw_data/sports/2.txt,https://news.google.com/__i/rss/rd/articles/CB...
2,2,sports,3,El fútbol es un cuento - La Voz de Galicia,./Datos/Raw_data/sports/3.txt,https://news.google.com/__i/rss/rd/articles/CB...
3,3,sports,4,Cavani y Giménez serán sancionados pero no con...,./Datos/Raw_data/sports/4.txt,https://news.google.com/__i/rss/rd/articles/CB...
4,4,sports,5,Suspendido por una tangana a puñetazos el Alme...,./Datos/Raw_data/sports/5.txt,https://news.google.com/__i/rss/rd/articles/CB...


In [193]:
clases, documentos = txt(df)

print(len(clases))
print(len(documentos))

//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//1.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//2.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//3.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//4.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//5.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//6.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//7.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//8.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//9.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//10.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//11.txt
//Users//joelpardo//Desktop//TextClassification//Datos//Raw_data//sports//12.txt
//Users//joelpardo//Desktop//TextClas

In [194]:
#Comprobamos los tipos
print(type(df))
# print(type(df_gen))

<class 'pandas.core.frame.DataFrame'>


In [195]:
df["docs"] = documentos

In [196]:
#Miramos los nombres
df.columns

Index(['index', 'category', 'n_doc', 'title', 'path', 'link', 'docs'], dtype='object')

In [197]:
#Generamos un duplicado para seleccionar lo que queremos
df2 = df.loc[:,['title', 'category','docs']]

#Eliminamos los na
df2=df2.dropna()

In [198]:
#Comprobamos cuantas fila con na hemos eliminado
print(df.shape)
print(df2.shape)

#Ninguna

(240, 7)
(240, 3)


In [199]:
#Comprobamos los valores de la columna category
# clases = pd.unique(df2['category']).tolist()
print(clases)


['sports', 'health', 'science', 'politics']


In [200]:
#Vamos a ver que clases tienen más peso
frecuencias = df2.groupby('category').count()
frecuencias = frecuencias.sort_values('title',ascending=False)

#Visualizamos las clases con mas peso (todas)
frecuencias.head()

,title,docs
category,,
health,60,60
politics,60,60
science,60,60
sports,60,60


Como podemos observar, todas las clases tienen el mismo peso, es decir, tienen el mismo numero de noticas. Tenemos un conjunto de datos homogeneo para el cual hacer la prediccion.

## 3. PRE-PROCESADO
Realizaremos el siguiente procesado:
- Separar el texto en *tokens*
- Eliminar los *tokens* de tipo *stop-word*, signos de puntuación, signos especiales o espacios
- Lematizar el texto
- Introducimos un espacio después de determinados signos de puntuación (".", "?", "%") para que el tokenizado sea correcto
- Filtramos los *tokens* con una longitud de 1



In [201]:
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 1.9 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [202]:
# https://spacy.io/models/es
nlp = spacy.load("es_core_news_lg") #Mejor modelo optimizado para la CPU

#definimos función de normalizado
def normaliza(texto):
    texto = re.sub(r"(\.)|(\?)|(\%)", r"\1 |\2 |\3 ", texto) #añadimos un espacio después de "." y "?"
    doc = nlp(texto)
    tokens = [t for t in doc if
                        len(t.text)>1 and
                       not t.is_stop and
                       not t.is_space and
                       not t.is_punct]#filtramos los tokens que nos interesan
    palabras = []
    for t in tokens:
        palabras.append(t.lemma_) #añadimos lema
    salida = ' '.join(palabras)#junta todos los tokens en un string
    
    return salida

#funcion para quitar acentos y caracteres no ASCII:

def remove_accents(input_str):     
    nfkd_form = unicodedata.normalize('NFKD', input_str)     
    only_ascii = nfkd_form.encode('ASCII', 'ignore')     
    return only_ascii.decode("utf-8", 'ignore')


In [203]:
#Comprobamos que funciona con el elemento 0 de noticia ("noticia")
print(df['docs'][0])
print("-"*140)
normaliza(df['docs'][0])

El lateral neerlands regal dos asistencias, marc el 1-3... y el delantero azulgrana bail con la defensa estadounidense para clasificar a la Oranje a los cuartos de final del Mundial
"En Pases Bajos aprend mucho, tom muchos conceptos de ftbol". Gregg Berhalter, seleccionador de Estados Unidos, asegur en la previa del partido que se inspir en el ftbol neerlands para fijar las bases de su ideario como entrenador. Unos conceptos futbolsticos que se dejaron ver en los primeros compases del partido, hasta que Memphis Depay decidi romper el espejismo impuesto por la caballera yankee.
Averiguaron la forma en la que tenan que atacar a Pases Bajos para desarmar su entramado defensivo pero Pulisic, que no tuvo su da, no consigui mandar al fondo de la red ni una de las dos ocasiones que tuvo en la primera parte. Una de ellas tardar mucho tiempo en olvidarla. 
Instaurados en el bloque bajo, los hombres de Louis van Gaal dieron una autntica leccin de cmo un equipo tiene que atacar el espacio con sus

'lateral neerlands regal asistencia marc 1-3 delantero azulgrana bail defensa estadounidense clasificar Oranje cuarto Mundial Pases Bajos aprend tom concepto ftbol Gregg Berhalter seleccionador Unidos asegur previa partido inspir ftbol neerlands fijar base ideario entrenador concepto futbolstico dejar compás partido Memphis Depay decidi romper espejismo impuesto caballera yankee averiguar forma tenar atacar Pases Bajos desarmar entramado defensivo Pulisic consigui mandar fondo red ocasión tardar tiempo olvidar él Instaurados bloque hombre Louis Gaal autntica leccin cmo equipo atacar espacio contra 10 bajar recibir liberar función centrocampista capaz llegar rea generar peligro conjunto neerlands caz rea Depay perdon certificar soccer todava est lejos modelo ftbol Pases Bajos cabalgada Dumfries ms suficiente romper defensa Unidos mand Depay fondo red tercer pase muerte lateral cort Adams problema apareci Blind firmar 2-0 justo descanso acción calcada ponar Oranje pie cuarto Mundial Qata

In [204]:
#Comprobamos que funciona con el elemento 2 de noticia ("docs")
print(df['docs'][2])
print("-"*140)
normaliza(df['docs'][2])

El ftbol es un cuento. Ya lo dijo ese Shakespeare futbolstico que es Gary Lineker, buen goleador y estrella de la tele: El ftbol es un juego muy simple. 22 hombres persiguen un baln durante 90 minutos y al final siempre ganan los alemanes. La frase ha envejecido mal. Las mujeres tambin corren detrs de la pelota y arbitran (aunque sea a cuentagotas). Los alemanes ya no le ganan ni a Japn (nosotros tampoco). Y ese juego simple puede ser un tostn. Ms que el ftbol, nos gusta el negocio del ftbol, esa fbrica de millonarios, algunos por lo civil y otros por lo criminal, como dira Luis Aragons. Nos gustan su cultura, sus batallas, el cuento del ftbol.
No est tan mal un Mundial en Navidad, al margen de la tropela de la FIFA y Catar. Con las mil y una noches persas y las luces del desierto y de las calles de Vigo. Las jvenes iranes jugando a tirarles el turbante a los ayatols. Los vecinos de la calle Prncipe contando ovejas viendo pasar el carricoche de la montaa rusa por la ventana.
Infantino 

'ftbol cuento Shakespeare futbolstico Gary Lineker goleador estrella tele ftbol juego simple 22 hombre perseguir baln 90 minuto ganar alemán frase envejecer mujer tambin correr detrs pelota arbitrar cuentagotas alemán ganar Japn juego simple tostn Ms ftbol gustar negocio ftbol fbrica millonario civil criminal dira Luis Aragons gustar cultura batalla cuento ftbol est Mundial Navidad margen tropela FIFA Catar mil noche persa luz desierto calle Vigo jvenes iranes jugar tirar él turbante ayatols vecino calle Prncipe contar oveja ver pasar carricoche montaa ruso ventana Infantino Gargamel Cristiano madrastra mirar espejo Luis Enrique principito arrogante Blatter Doctor Claw malo rostro inspector Gadget acariciar gatito Messi Messi Luis Surez llorar paisito gallego seleccionador Camern ir casa dejar posteridad teora peligro ests peligro realidad ests peligro ests peligro realmente ests peligro Navidad ao inaugurar Black Friday Caballero Catar-Ecuador Doha cuento'

In [205]:
#Creacion de corpus y labels 
corpus = df['docs'].tolist() + df['title'].tolist()
labels_posibles = pd.unique(df2['category']).tolist()
labels = df['category'].tolist() + df['category'].tolist() 

In [206]:
#Vemos cuantas observaciones en total tenemos.
print(f'\nTamaño  TOTAL: {len(corpus)}')
print(f'Tamaño de clase TOTAL: {len(labels)}')



Tamaño  TOTAL: 480
Tamaño de clase TOTAL: 480


In [207]:
#Seleccionamos 5000 muestras del cojunto de datos total
# corpus_sm, labels_sm = resample(corpus, labels, n_samples=5000, replace=False) 

In [208]:
#Separacion del conjunto de datos P5
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus, labels, test_size = 0.30, shuffle = False)

In [209]:
# COMPLETAR
print(f'\nTamaño de TRAIN: {len(train_corpus)}')
print(f'Tamaño de clase TRAIN: {len(train_labels)}')

print(f'\nTamaño de TEST: {len(test_corpus)}')
print(f'Tamaño de clase TEST: {len(test_labels)}')


Tamaño de TRAIN: 336
Tamaño de clase TRAIN: 336

Tamaño de TEST: 144
Tamaño de clase TEST: 144


In [210]:
#Normalizamos los docss y los guardamos como una lista en lugar de generator porque 
#lo tenemos que múltiples veces y no queremos tener que normalizar todo el corpus cada vez.
norm_train_corpus = list(map(normaliza, train_corpus))

In [211]:
norm_test_corpus = list(map(normaliza, test_corpus))

## 4. EXTRACCION DE CARACTERISTICAS
Instanciamos los vectorizadores para obtener las características BoW y TF-IDF.  
Usamos el parámetro max_df=0.9 para eliminar los stop-words como las palabras que aparecen al menos en el 90% de los documentos y el parámetro min_df=0.01 para eliminar las palabras que no aparecen al menos en un 1% de los documentos.\

Usamos el modelo `TfidfTransformer` para calcular la matriz TF-IDF a partir del BoW y no tener que repetir todo el entrenamiento.

Para calcular los modelos basados en WV usamos el modelo gloVe pre-entrenado en `spaCy`. Calculamos dos modelos basados en word-vectors:  
* El vector promedio de los WV de todos los tokens con el mismo peso para todas las palabras.  
* Ponderando el WV de cada palabra por el término de frecuencia inversa de documento (IDF).  

Definimos las funciones para calcular estas dos matrices de características:

In [212]:
#BoW
bow_vectorizer = CountVectorizer(min_df=0.01, max_df=0.9)

#Tf-idf
tfidf_vectorizer = TfidfTransformer()

#Funciones de WV.
def averaged_word_vectorizer(corpus):
    '''Aplica la función de cálculo del WE promedio a todos los
    documentos del corpus (cada doc es una lista de tokens)'''
    features = [nlp(doc).vector
                    for doc in corpus]
    return np.array(features)

def tfidf_wtd_avg_word_vectors(doc, word_tfidf_map):
    '''Aplica la función de cálculo del WE ponderado por TF-IDF
    a un documento (como lista de tokens)'''
    tokens = doc.split()

    feature_vector = np.zeros((nlp.vocab.vectors_length,),dtype="float64")
    wts = 0.      
    for word in tokens:
        if nlp.vocab[word].has_vector and word_tfidf_map.get(word, 0): #sólo considera palabras conocidas
            weighted_word_vector = word_tfidf_map[word] * nlp.vocab[word].vector
            wts = wts + 1
            feature_vector = np.add(feature_vector, weighted_word_vector)
    if wts:
        feature_vector = np.divide(feature_vector, wts)
        
    return feature_vector
    
def tfidf_weighted_averaged_word_vectorizer(corpus, word_tfidf_map):
    '''Aplica la función de cálculo del WE ponderado por TF-IDF a todos los
    documentos del corpus (cada doc es una lista de tokens)'''                                       
    features = [tfidf_wtd_avg_word_vectors(doc, word_tfidf_map)
                    for doc in corpus]
    return np.array(features)

In [213]:
# características bag of words
bow_train_features = bow_vectorizer.fit_transform(norm_train_corpus)  
bow_test_features = bow_vectorizer.transform(norm_test_corpus) 

# características tfidf (a partir del BoW)
tfidf_train_features = tfidf_vectorizer.fit_transform(bow_train_features)
tfidf_test_features = tfidf_vectorizer.transform(bow_test_features)    

# características averaged word vector
avg_wv_train_features = averaged_word_vectorizer(norm_train_corpus)                
avg_wv_test_features = averaged_word_vectorizer(norm_test_corpus)      

# características tfidf weighted averaged word vector
word_tfidf_map = {key:value for (key, value) in zip(bow_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_)}

tfidf_wv_train_features = tfidf_weighted_averaged_word_vectorizer(norm_train_corpus, word_tfidf_map)

tfidf_wv_test_features = tfidf_weighted_averaged_word_vectorizer(norm_test_corpus, word_tfidf_map)

In [214]:
bow_train_features.shape

(336, 3142)

In [215]:
tfidf_train_features.shape

(336, 3142)

In [216]:
avg_wv_train_features.shape

(336, 300)

In [217]:
tfidf_wv_train_features.shape

(336, 300)

## 5. CLASIFICADOR
Aplicamos distintos clasificadores a cada modelo para ver cuál funciona mejor con nuestros datos. Primero definimos unas funciones para entrenar y medir el rendimiento de los clasificadores:

In [218]:
def get_metrics(true_labels, predicted_labels):
    """Calculamos distintas métricas sobre el
    rendimiento del modelo. Devuelve un diccionario
    con los parámetros medidos"""
    
    return {
        'Accuracy': np.round(
                        metrics.accuracy_score(true_labels, 
                                               predicted_labels),
                        3),
        'Precision': np.round(
                        metrics.precision_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'Recall': np.round(
                        metrics.recall_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3),
    'F1 Score': np.round(
                        metrics.f1_score(true_labels, 
                                               predicted_labels,
                                               average='weighted',
                                               zero_division=0),
                        3)}
                        

def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    """Función que entrena un modelo de clasificación sobre
    un conjunto de entrenamiento, lo aplica sobre un conjunto
    de test y devuelve la predicción sobre el conjunto de test
    y las métricas de rendimiento"""
    # genera modelo    
    classifier.fit(train_features, train_labels)
    # predice usando el modelo sobre test
    predictions = classifier.predict(test_features) 
    # evalúa rendimiento de la predicción   
    metricas = get_metrics(true_labels=test_labels, 
                predicted_labels=predictions)
    return predictions, metricas     

Vamos a entrenar sobre el conjunto de train y evaluamos en el conjunto de test. Guardamos métricas en una lista y resultados en otra para mostrar resumen.

In [219]:
modelLR = LogisticRegression(solver='liblinear')
modelNB = GaussianNB()
modelSVM = SGDClassifier(loss='hinge', max_iter=1000)
modelRBFSVM = SVC(gamma='scale', C=2)

modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('Linear SVM', modelSVM),
           ('Gauss kernel SVM', modelRBFSVM)]

metricas = []
resultados = []

# Modelos con características BoW
bow_train_features_ = bow_train_features.toarray()
bow_test_features_ = bow_test_features.toarray()
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=bow_train_features_,
                                           train_labels=train_labels,
                                           test_features=bow_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} BoW'
    resultados.append(prediccion)
    metricas.append(metrica)
    
# Modelos con características TF-IDF
tfidf_train_features_ = tfidf_train_features.toarray()
tfidf_test_features_ = tfidf_test_features.toarray()
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_train_features_,
                                           train_labels=train_labels,
                                           test_features=tfidf_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} tfidf'
    resultados.append(prediccion)
    metricas.append(metrica)

# Modelos con características averaged word vectors
avg_wv_train_features_ = avg_wv_train_features
avg_wv_test_features_ = avg_wv_test_features
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=avg_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} averaged'
    resultados.append(prediccion)
    metricas.append(metrica)

# Modelos con características tfidf weighted averaged word vectors
tfidf_wv_train_features_ = tfidf_wv_train_features
tfidf_wv_test_features_ = tfidf_wv_test_features
for m, clf in modelos:
    prediccion, metrica = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=tfidf_wv_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{m} tfidf wv'
    resultados.append(prediccion)
    metricas.append(metrica)

Conviertimos la lista de métricas en un DataFrame para observar sus valores:

In [220]:
metricas = pd.DataFrame(metricas)
metricas

,Accuracy,Precision,Recall,F1 Score,modelo
0,0.097,0.097,0.097,0.097,Logistic Regression BoW
1,0.639,0.842,0.639,0.682,Naive Bayes BoW
2,0.153,0.883,0.153,0.113,Linear SVM BoW
3,0.000,0.000,0.000,0.000,Gauss kernel SVM BoW
4,0.222,0.955,0.222,0.304,Logistic Regression tfidf
5,0.861,0.951,0.861,0.899,Naive Bayes tfidf
6,0.299,0.905,0.299,0.343,Linear SVM tfidf
7,0.285,0.929,0.285,0.332,Gauss kernel SVM tfidf
8,0.812,0.863,0.812,0.826,Logistic Regression averaged
9,0.153,0.475,0.153,0.132,Naive Bayes averaged


Ordenamos las métricas por `accuracy` y muestra el mejor resultado:

In [221]:
metricas = metricas.sort_values('Accuracy',ascending=False)
metricas.head(1)

,Accuracy,Precision,Recall,F1 Score,modelo
5,0.861,0.951,0.861,0.899,Naive Bayes tfidf


Mejoramos el `accuracy` a partir del juego de parametros

In [222]:
#https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
#Funcion de gamma
gamma = ['scale', 'auto']

metricas = []
resultados = []


for i in gamma:
    modelRBFSVM = SVC(gamma=i, C=2)
    prediccion, metrica = train_predict_evaluate_model(classifier=modelRBFSVM,
                                           train_features=avg_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features_,
                                           test_labels=test_labels)
    metrica['modelo']=f'{i} – Gauss kernel SVM averaged'
    resultados.append(prediccion)
    metricas.append(metrica)

Conviertimos la lista de métricas en un DataFrame para observar sus valores:

In [223]:
metricas = pd.DataFrame(metricas)
metricas

,Accuracy,Precision,Recall,F1 Score,modelo
0,0.424,0.851,0.424,0.465,scale – Gauss kernel SVM averaged
1,0.549,0.869,0.549,0.588,auto – Gauss kernel SVM averaged


Ordenamos las métricas por `accuracy` y muestra el mejor resultado:

In [224]:
metricas = metricas.sort_values('Accuracy',ascending=False)
metricas.head(1)

,Accuracy,Precision,Recall,F1 Score,modelo
1,0.549,0.869,0.549,0.588,auto – Gauss kernel SVM averaged


Dividimos el conjunto total de muestras (`corpus` y `labels`) en entrenamiento y test (30%) y entrenamos el mejor modelo obtenido, para ver si se mejoran los resultados:

In [225]:
## MEJOR MODELO
train_corpus, test_corpus, train_labels, test_labels = train_test_split(corpus, labels, test_size = 0.30)


In [226]:
#Normalizamos
norm_train_corpus = list(map(normaliza, train_corpus))
norm_test_corpus = list(map(normaliza, test_corpus))



In [227]:
# Características averaged word vector
avg_wv_train_features = averaged_word_vectorizer(norm_train_corpus)                
avg_wv_test_features = averaged_word_vectorizer(norm_test_corpus)      
avg_wv_train_features_ = avg_wv_train_features
avg_wv_test_features_ = avg_wv_test_features

In [228]:
# Modelos con características
modelRBFSVM = SVC(gamma='scale', C=2)
prediccion, metrica = train_predict_evaluate_model(classifier=modelRBFSVM,
                                           train_features=avg_wv_train_features_,
                                           train_labels=train_labels,
                                           test_features=avg_wv_test_features_,
                                           test_labels=test_labels)



In [229]:
print('Modelo Gauss kernel SVM averaged con gamma scale: ')
for i in metrica:
    print('- ',i, ': ', metrica[i])

print('\nPredicciones:')
print(prediccion)

Modelo Gauss kernel SVM averaged con gamma scale: 
-  Accuracy :  0.896
-  Precision :  0.896
-  Recall :  0.896
-  F1 Score :  0.896

Predicciones:
['sports' 'politics' 'health' 'science' 'science' 'sports' 'science'
 'politics' 'sports' 'science' 'science' 'politics' 'science' 'science'
 'health' 'health' 'science' 'politics' 'science' 'sports' 'health'
 'science' 'health' 'sports' 'sports' 'health' 'sports' 'health' 'sports'
 'science' 'sports' 'politics' 'sports' 'health' 'health' 'health'
 'politics' 'science' 'science' 'politics' 'health' 'health' 'politics'
 'sports' 'health' 'politics' 'sports' 'sports' 'sports' 'health'
 'science' 'politics' 'politics' 'sports' 'science' 'science' 'politics'
 'sports' 'politics' 'science' 'sports' 'sports' 'science' 'health'
 'health' 'sports' 'health' 'health' 'science' 'health' 'health' 'science'
 'health' 'health' 'science' 'politics' 'sports' 'politics' 'science'
 'politics' 'politics' 'health' 'politics' 'health' 'science' 'sports'
 'spor

In [230]:
pd.unique(prediccion)

array(['sports', 'politics', 'health', 'science'], dtype='<U8')